# 🌲 Forest Cover Type Prediction

### Kaggle Competition Solution using Stacking & Blending Ensemble

This notebook presents a robust machine learning pipeline for the [Forest Cover Type Prediction](https://www.kaggle.com/c/forest-cover-type-prediction) competition. The goal is to predict the forest cover type (7 classes) based on cartographic variables.

**Key Techniques:**
- Advanced Feature Engineering (distance metrics, terrain analysis, domain features)
- Target Encoding with Cross-Validation (leak-free)
- Gradient Boosting Ensemble: LightGBM, XGBoost, CatBoost
- Two Ensemble Strategies: Stacking (meta-learner) & Blending (weighted average)

---


## 1. Setup & Imports


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import gc

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier

print("✓ All libraries loaded successfully")


In [ ]:
# Configuration
N_FOLDS = 7
SEED = 42


## 2. Data Loading

The dataset contains cartographic variables for 30m x 30m cells from Roosevelt National Forest in Colorado. Features include:
- **Elevation**: Elevation in meters
- **Aspect**: Azimuth angle (0-360°)
- **Slope**: Slope in degrees
- **Distances**: Horizontal/vertical distance to hydrology, roadways, fire points
- **Hillshade**: Illumination values at 9am, noon, and 3pm (summer solstice)
- **Wilderness Areas**: 4 binary columns indicating wilderness area designation
- **Soil Types**: 40 binary columns for soil type classification


In [ ]:
def load_data():
    """
    Load train, test and sample submission files.
    Searches common paths: current directory, Desktop, Kaggle input.
    """
    search_paths = [
        Path.cwd(),
        Path.home() / "Desktop",
        Path("/kaggle/input/forest-cover-type-prediction")
    ]
    
    for path in search_paths:
        if not path.exists():
            continue
            
        train_path = path / "train.csv"
        test_path = path / "test-full.csv" if (path / "test-full.csv").exists() else path / "test.csv"
        submission_path = path / "full_submission.csv" if (path / "full_submission.csv").exists() else path / "sample_submission.csv"
        
        if all(f.exists() for f in [train_path, test_path, submission_path]):
            print(f"✓ Data found in: {path}")
            return pd.read_csv(train_path), pd.read_csv(test_path), pd.read_csv(submission_path)
    
    raise FileNotFoundError("CSV files (train/test/submission) not found.")


In [ ]:
train_df, test_df, sample_submission = load_data()

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"\nTarget distribution:")
print(train_df["Cover_Type"].value_counts().sort_index())


## 3. Feature Engineering

We create domain-specific features that capture important relationships in the data:

### Distance Features
- **Euclidean & Manhattan distances** to hydrology sources
- **Distance ratios** between hydrology, roads, and fire points

### Terrain Features
- **Elevation interactions** with vertical distance to hydrology
- **Hillshade statistics**: mean and range across time points
- **Aspect decomposition**: North-South and East-West components (cosine/sine transform)

### Categorical Features
- **Wilderness Area ID**: Collapsed from binary columns
- **Soil Type ID**: Collapsed from 40 binary columns
- **Climate Zone**: Derived from soil classification
- **Stony Soil Indicator**: Based on domain knowledge of soil descriptions


In [ ]:
def create_features(df):
    """
    Create advanced features from raw data.
    Returns processed dataframe and list of categorical feature names.
    """
    df = df.copy()
    
    # --- Distance Features ---
    # Euclidean and Manhattan distance to hydrology
    df['Hydro_Euclidean'] = np.sqrt(
        df['Horizontal_Distance_To_Hydrology']**2 + 
        df['Vertical_Distance_To_Hydrology']**2
    )
    df['Hydro_Manhattan'] = (
        np.abs(df['Horizontal_Distance_To_Hydrology']) + 
        np.abs(df['Vertical_Distance_To_Hydrology'])
    )
    
    # --- Elevation Interactions ---
    df['Elev_minus_Hydro_Vert'] = df['Elevation'] - df['Vertical_Distance_To_Hydrology']
    df['Elev_plus_Hydro_Vert'] = df['Elevation'] + df['Vertical_Distance_To_Hydrology']
    
    # --- Distance Ratios ---
    eps = 1e-6  # Small constant to avoid division by zero
    df['Hydro_Road_Ratio'] = (
        df['Horizontal_Distance_To_Hydrology'] / 
        (df['Horizontal_Distance_To_Roadways'] + eps)
    )
    df['Hydro_Fire_Ratio'] = (
        df['Horizontal_Distance_To_Hydrology'] / 
        (df['Horizontal_Distance_To_Fire_Points'] + eps)
    )
    df['Road_Fire_Ratio'] = (
        df['Horizontal_Distance_To_Roadways'] / 
        (df['Horizontal_Distance_To_Fire_Points'] + eps)
    )
    
    # --- Hillshade Statistics ---
    hillshade_cols = ["Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm"]
    df["Hillshade_Mean"] = df[hillshade_cols].mean(axis=1)
    df["Hillshade_Range"] = df[hillshade_cols].max(axis=1) - df[hillshade_cols].min(axis=1)
    
    # --- Aspect Decomposition (Circular Feature) ---
    aspect_rad = np.deg2rad(df["Aspect"])
    df["North_South_Component"] = np.cos(aspect_rad)
    df["East_West_Component"] = np.sin(aspect_rad)
    
    # --- Categorical ID Features ---
    wilderness_cols = [c for c in df.columns if "Wilderness_Area" in c]
    soil_cols = [c for c in df.columns if "Soil_Type" in c]
    
    if wilderness_cols:
        df["Wilderness_ID"] = np.argmax(df[wilderness_cols].values, axis=1) + 1
    if soil_cols:
        df["Soil_ID"] = np.argmax(df[soil_cols].values, axis=1) + 1
    
    # --- Domain-Specific Features ---
    # Climate zone derived from soil type classification
    df['Climate_Zone'] = df['Soil_ID'].apply(lambda x: x // 1000)
    
    # Stony soils based on soil type descriptions from documentation
    stony_soils = [3, 4, 5, 6, 9, 10, 11, 12, 13, 18, 22, 24, 26, 27, 28, 29, 
                   30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
    df['Is_Stony'] = df['Soil_ID'].isin(stony_soils).astype(int)
    
    # --- Interaction Features ---
    df["Elev_x_Wilderness"] = df["Elevation"] * df["Wilderness_ID"]
    df["Slope_x_Wilderness"] = df["Slope"] * df["Wilderness_ID"]
    
    # Define categorical features for target encoding
    categorical_features = ["Wilderness_ID", "Soil_ID", "Climate_Zone"]
    
    # --- Clean up: Handle infinities and NaN ---
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0).astype(np.float32)
    
    return df, categorical_features


## 4. Target Encoding

Target encoding replaces categorical values with the mean target value for that category. To prevent data leakage, we use cross-validation target encoding with smoothing.


In [ ]:
def target_encode_cv(X_train, y_train, X_test, categorical_features, n_splits=7, smoothing=30):
    """
    Apply target encoding with cross-validation to prevent leakage.
    
    Parameters:
    -----------
    X_train : DataFrame - Training features
    y_train : Series - Training target
    X_test : DataFrame - Test features
    categorical_features : list - Columns to encode
    n_splits : int - Number of CV folds
    smoothing : float - Smoothing parameter (higher = more regularization)
    
    Returns:
    --------
    X_train, X_test with new target-encoded features
    """
    X_train = X_train.copy()
    X_test = X_test.copy()
    global_mean = y_train.mean()
    
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    for feature in categorical_features:
        if feature not in X_train.columns:
            continue
            
        # Initialize arrays
        te_train = np.zeros(len(X_train))
        te_test_accumulated = np.zeros(len(X_test))
        
        for train_idx, val_idx in kf.split(X_train, y_train):
            # Compute statistics on training fold only
            fold_data = pd.DataFrame({
                feature: X_train.iloc[train_idx][feature],
                'target': y_train.iloc[train_idx]
            })
            stats = fold_data.groupby(feature)['target'].agg(['mean', 'count'])
            
            # Apply smoothing: blend category mean with global mean
            smoothed_means = (
                (stats['count'] * stats['mean'] + smoothing * global_mean) / 
                (stats['count'] + smoothing)
            )
            
            # Encode validation fold
            te_train[val_idx] = X_train.iloc[val_idx][feature].map(smoothed_means).fillna(global_mean).values
            
            # Accumulate test encodings
            te_test_accumulated += X_test[feature].map(smoothed_means).fillna(global_mean).values
        
        # Add encoded features
        X_train[f"{feature}_TE"] = te_train
        X_test[f"{feature}_TE"] = te_test_accumulated / n_splits
    
    return X_train, X_test


## 5. Data Preprocessing


In [ ]:
# Extract target (convert to 0-indexed for sklearn)
y = train_df["Cover_Type"] - 1

# Apply feature engineering
print("Applying feature engineering...")
X, categorical_features = create_features(train_df.drop(columns=["Cover_Type", "Id"]))
X_test, _ = create_features(test_df.drop(columns=["Id"]))

# Apply target encoding
print("Applying target encoding...")
X, X_test = target_encode_cv(X, y, X_test, categorical_features, n_splits=N_FOLDS)

# Ensure same columns in train and test
shared_columns = list(set(X.columns) & set(X_test.columns))
X = X[shared_columns]
X_test = X_test[shared_columns]

print(f"\n✓ Final shapes: Train={X.shape}, Test={X_test.shape}")


## 6. Model Training

We train three gradient boosting models with optimized hyperparameters:
- **LightGBM**: Fast, efficient, good with categorical features
- **XGBoost**: Robust, handles regularization well
- **CatBoost**: Excellent with categorical data, symmetric trees


In [ ]:
def train_model(X, y, X_test, params, model_type='lgbm', seed=42):
    """
    Train a model using K-Fold cross-validation.
    
    Returns:
    --------
    oof_preds : Out-of-fold probability predictions (for stacking)
    test_preds : Averaged test predictions
    """
    n_classes = 7
    oof_preds = np.zeros((len(X), n_classes))
    test_preds = np.zeros((len(X_test), n_classes))
    
    kf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=seed)
    
    for fold, (train_idx, val_idx) in enumerate(tqdm(kf.split(X, y), total=N_FOLDS, desc=f"{model_type.upper()}")):
        X_train_fold = X.iloc[train_idx]
        X_val_fold = X.iloc[val_idx]
        y_train_fold = y.iloc[train_idx]
        y_val_fold = y.iloc[val_idx]
        
        if model_type == 'lgbm':
            model = lgb.LGBMClassifier(**params)
            model.fit(
                X_train_fold, y_train_fold,
                eval_set=[(X_val_fold, y_val_fold)],
                callbacks=[lgb.early_stopping(150, verbose=False)]
            )
        elif model_type == 'xgb':
            model = xgb.XGBClassifier(**params)
            model.fit(
                X_train_fold, y_train_fold,
                eval_set=[(X_val_fold, y_val_fold)],
                verbose=False
            )
        elif model_type == 'cat':
            model = CatBoostClassifier(**params)
            model.fit(
                X_train_fold, y_train_fold,
                eval_set=[(X_val_fold, y_val_fold)],
                use_best_model=True,
                verbose=False
            )
        
        # Store predictions
        oof_preds[val_idx] = model.predict_proba(X_val_fold)
        test_preds += model.predict_proba(X_test) / N_FOLDS
        
        # Clean up memory
        del model, X_train_fold, X_val_fold, y_train_fold, y_val_fold
        gc.collect()
    
    # Calculate CV score
    cv_score = accuracy_score(y, np.argmax(oof_preds, axis=1))
    print(f"  {model_type.upper()} CV Score: {cv_score:.5f}")
    
    return oof_preds, test_preds


In [ ]:
# Hyperparameters (tuned for high performance)

params_lgbm = {
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'random_state': SEED,
    'n_estimators': 2000,
    'learning_rate': 0.02,
    'num_leaves': 100,
    'max_depth': 12,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'min_child_samples': 20,
    'n_jobs': -1
}

params_xgb = {
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'random_state': SEED,
    'n_estimators': 2000,
    'learning_rate': 0.02,
    'max_depth': 10,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0.1,
    'tree_method': 'hist',
    'early_stopping_rounds': 150,
    'n_jobs': -1
}

params_cat = {
    'loss_function': 'MultiClass',
    'random_seed': SEED,
    'iterations': 3000,
    'learning_rate': 0.03,
    'depth': 10,
    'l2_leaf_reg': 5,
    'bagging_temperature': 0.8,
    'od_type': 'Iter',
    'od_wait': 150,
    'thread_count': -1
}


In [ ]:
# Train all models
print("Training models...\n")

all_oof = {}
all_preds = {}

# LightGBM
oof_lgbm, preds_lgbm = train_model(X, y, X_test, params_lgbm, 'lgbm')
all_oof['lgbm'] = oof_lgbm
all_preds['lgbm'] = preds_lgbm

# XGBoost
oof_xgb, preds_xgb = train_model(X, y, X_test, params_xgb, 'xgb')
all_oof['xgb'] = oof_xgb
all_preds['xgb'] = preds_xgb

# CatBoost
oof_cat, preds_cat = train_model(X, y, X_test, params_cat, 'cat')
all_oof['cat'] = oof_cat
all_preds['cat'] = preds_cat

print("\n✓ All models trained successfully")


## 7. Ensemble Strategy 1: Stacking

Stacking uses a meta-learner (Logistic Regression) trained on the out-of-fold predictions from base models. This learns the optimal way to combine model outputs.


In [ ]:
# Concatenate OOF predictions from all models
oof_stacked = np.hstack(list(all_oof.values()))
preds_stacked = np.hstack(list(all_preds.values()))

print(f"Stacked features shape: {oof_stacked.shape}")

# Train meta-learner
meta_model = LogisticRegression(multi_class='multinomial', max_iter=1000)
meta_model.fit(oof_stacked, y)

# Generate predictions
oof_final_stacking = meta_model.predict_proba(oof_stacked)
preds_final_stacking = meta_model.predict_proba(preds_stacked)

# Evaluate
stacking_score = accuracy_score(y, np.argmax(oof_final_stacking, axis=1))
print(f"\n{'='*50}")
print(f"STACKING CV SCORE: {stacking_score:.6f}")
print(f"{'='*50}")


## 8. Ensemble Strategy 2: Blending (Weighted Average)

Blending combines model predictions using fixed weights. Weights are assigned based on individual model performance.


In [ ]:
# Blending weights (based on typical model performance)
WEIGHTS = {
    'lgbm': 0.45,
    'cat': 0.35,
    'xgb': 0.20
}

print(f"Blending weights: {WEIGHTS}")

# Weighted average of predictions
oof_final_blending = (
    all_oof['lgbm'] * WEIGHTS['lgbm'] +
    all_oof['cat'] * WEIGHTS['cat'] +
    all_oof['xgb'] * WEIGHTS['xgb']
)

preds_final_blending = (
    all_preds['lgbm'] * WEIGHTS['lgbm'] +
    all_preds['cat'] * WEIGHTS['cat'] +
    all_preds['xgb'] * WEIGHTS['xgb']
)

# Evaluate
blending_score = accuracy_score(y, np.argmax(oof_final_blending, axis=1))
print(f"\n{'='*50}")
print(f"BLENDING CV SCORE: {blending_score:.6f}")
print(f"{'='*50}")


## 9. Generate Submissions


In [ ]:
# Create submission files

# Stacking submission
submission_stacking = sample_submission.copy()
submission_stacking["Cover_Type"] = np.argmax(preds_final_stacking, axis=1) + 1
stacking_filename = f"submission_stacking_{stacking_score:.5f}.csv"
submission_stacking.to_csv(stacking_filename, index=False)
print(f"✓ Stacking submission saved: {stacking_filename}")

# Blending submission
submission_blending = sample_submission.copy()
submission_blending["Cover_Type"] = np.argmax(preds_final_blending, axis=1) + 1
blending_filename = f"submission_blending_{blending_score:.5f}.csv"
submission_blending.to_csv(blending_filename, index=False)
print(f"✓ Blending submission saved: {blending_filename}")


## Summary

| Ensemble Method | CV Score |
|-----------------|----------|
| Stacking (Logistic Regression Meta-Learner) | See output above |
| Blending (Weighted Average) | See output above |

**Tips for improving score:**
- Tune blending weights using validation data
- Add more diverse base models (Neural Networks, ExtraTrees)
- Feature selection using permutation importance
- Pseudo-labeling with high-confidence test predictions
